# Ajuste do modelo "bootcamp"

Este notebook tem o objetivo de fazer os ajutes no modelo de dados que foi utilizado no Trabalho Prático do Módulo 3 (Coleta e Obtenção de Dados) do Bootcamp de Analista de Dados do IGTI.

Para isso, iremos  partir do princípio que o modelo foi criado conforme o arquivo fornecido para realização do trabalho prático, com as seguintes tabelas:

![MER](https://i.imgur.com/62RwXMS.png)

O objetivo não é praticar as metodologias que foram apresentadas nas aulas nem realizar o trabalho prático, mas sim fazer os ajustes no modelos de dados utilizando apenas o **Pandas** e o **SQLAlchemy**.

### Importação das bibliotecas:
- **pandas**: para manibulação dos dataframes (leitura de arquivoe e gravação em banco de dados)
- **sqlalchemy**: para gravação dos dataframes no MySQL

In [1]:
import pandas as pd
from sqlalchemy import create_engine

### Conexão com MySQL
Conecão com o banco de dados utilizando o **SQLAlchemy**.

Parâmetros da string de conexão:
- usr: usuário
- pwd: senha
- host: nome do servidor
- db: nome do bando de dados

In [2]:
engine = create_engine('mysql+pymysql://{usr}:{pwd}@{host}/{db}'
                       .format(usr = 'igti', 
                               pwd = 'igti', 
                               host = 'localhost', 
                               db = 'bootcamp2'))

conn = engine.connect()

### Criação das Tabelas
As tabelas **CaracteristicasGerais** e **CaracteristicaGeralImovel** não estão no modelo inicial, então é necessário criá-las.

In [3]:
cmd = "CREATE TABLE IF NOT EXISTS caracteristicasgerais (\
            idcaracteristicasGerais int NOT NULL AUTO_INCREMENT,\
            dsccaracteristicasGerais varchar(150) NOT NULL,\
            PRIMARY KEY (idcaracteristicasGerais)\
        );"
conn.execute(cmd)

cmd = "CREATE TABLE IF NOT EXISTS caracteristicageralimovel (\
          idCaracteristicasImovel int NOT NULL AUTO_INCREMENT,\
          idcaracteristicasGerais int NOT NULL,\
          idImovel int NOT NULL,\
          temCaracteristica tinyint DEFAULT NULL,\
          PRIMARY KEY (idCaracteristicasImovel),\
          KEY fk_caracteristicasGerais_has_Imovel_Imovel1_idx (idImovel),\
          KEY fk_caracteristicasGerais_has_Imovel_caracteristicasGerais1_idx (idcaracteristicasGerais),\
          CONSTRAINT fk_ccaracteristicageralimovel_caracteristicasgerais FOREIGN KEY (idcaracteristicasGerais) REFERENCES caracteristicasgerais (idcaracteristicasGerais),\
          CONSTRAINT fk_caracteristicageralimovel_Imovel FOREIGN KEY (idImovel) REFERENCES imovel (idImovel)\
        );"
conn.execute(cmd)

### Inclusão Manual de Características
Nas vídeo-aulas a professora utiliza alguns métodos diferentes para realizas a inclusão das três primeiras categorias('Poertair 24 horas', 'Elevador' e 'Piscina'). Aqui será criado manualmente um DataFrame no Pandas para carregá-lo direto para a tabela.

Para utilizar o método to_sql do DataFrame é necessário passar Engine como parâmetro (não a Conexão).

In [4]:
# Criação de um dicionário com as informações
caracteristicas = {
    'idcaracteristicasGerais': [1, 2, 3],
    'dsccaracteristicasGerais': ["Portaria 24 horas", "Elevador", "Piscina"]
}

# Criação de um DataFrame à partir do dicionário
df_caracteristicas = pd.DataFrame(caracteristicas)

# Inclusão das características na tabela
df_caracteristicas.to_sql(name='caracteristicasgerais', con=engine, if_exists='append', index=False)

### Importação das demais Características
As demais características do imóveis estão em um arquivo CSV, então é possível importá-las para um DataFrame e então inserí-las diretamente na tabela.

In [5]:
# Importa do CSV
df_caracteristicas = pd.read_csv('arquivos/caracteristicasgerais.csv', encoding='latin1')

# Insere na tabela
df_caracteristicas.to_sql(name='caracteristicasgerais', con=engine, if_exists='append', index=False)

### Importação dos Estados
A lista dos estados está em um arquivo do Excel (.xlsx), então é possível importá-los para um DataFrama e então inserí-los diretamente na tabela.

In [6]:
# Importa do Excel
df_estados = pd.read_excel('arquivos/estados.xlsx')

# Insere na tabela
df_estados.to_sql(name='estado', con=engine, if_exists='append', index=False)

### Importação das Cidades
As lista das cidades está em um arquivo CSV, então é possível importá-las para um DataFrame e então inserí-las diretamente na tabela.

In [7]:
# Importa do CSV
df_cidades = pd.read_csv('arquivos/cidades.csv', encoding='latin1')

# Insere na tabela
df_cidades.to_sql(name='cidade', con=engine, if_exists='append', index=False)

### Importação dos Imóveis
Os dados dos imóveis estão em um arquivo TXT em formato de comandos INSERTs prontos para serem executados na tabela.

In [8]:
file_imoveis = 'arquivos/imoveis.txt'

# Abre e lê o conteúdo do arquivo
f = open(file_imoveis, "r")
lines = f.readlines() 

# Iteração sobre as linhas e execução dos comandos
for line in lines: 
    if line.strip() != '':
        conn.execute(line.strip())

### Importação do vínculo entre os Imóveis e suas Características
As informações dos relacionamentos entre os imóveis e suas características estão em um arquivo CSV, então é possível importá-las para um DataFrame e então inserí-las diretamente na tabela.

In [9]:
# Importa do CSV
df_caracteristicas_imovel = pd.read_csv('Arquivos/caracteristicaImovel.csv', encoding='latin1')

# Insere na tabela
df_caracteristicas_imovel.to_sql(name='caracteristicageralimovel', con=engine, if_exists='append', index=False)